## Importing Necessary Libraries

In [1]:
from __future__ import division

import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.models import load_model

Using Theano backend.


## Building the Network

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(1, 28, 28), data_format="channels_first"))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), data_format="channels_first"))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv2D(16, (9, 9), data_format="channels_first"))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(10, activation='sigmoid'))

sgd = SGD(lr=0.1)

model.compile(optimizer="rmsprop",
              loss='categorical_crossentropy',
              metrics=['accuracy'])


## Data Handling

In [4]:
path = "C:\work\ml\digit_train.csv"

df = pd.read_csv(path)

labels = df["label"]
data = df.iloc[:, 1:]

In [5]:
train_data = data.sample(frac=0.99, random_state=80)
test_data = data.drop(train_data.index)

train_labels = labels.sample(frac=0.99, random_state=80)
test_labels = labels.drop(train_labels.index)

one_hot_train_labels = np.array(pd.get_dummies(train_labels))
train_data = np.array([np.array([x.reshape(-1, 28)], ) for x in np.array(train_data)])

one_hot_test_labels = np.array(pd.get_dummies(test_labels))
test_data = np.array([np.array([x.reshape(-1, 28)], ) for x in np.array(test_data)])


## Model Training

In [7]:
epochs = 3

model.fit(train_data, one_hot_train_labels, verbose=1, epochs=epochs)


Epoch 1/3
41580/41580 [==============================] - 143s - loss: 0.5607 - acc: 0.8285   
Epoch 2/3
41580/41580 [==============================] - 246s - loss: 0.1356 - acc: 0.9646   
Epoch 3/3
41580/41580 [==============================] - 800s - loss: 0.1154 - acc: 0.9706   


## Save Model

In [8]:
model.save('kaggle_digit_recognition_model.h5')

## Load Model

In [2]:
model = load_model('kaggle_digit_recognition_model.h5')

In [6]:
score = model.evaluate(test_data, one_hot_test_labels, verbose=1)
score

416/420 [============================>.] - ETA: 0s

[0.064149459461415453, 0.98809523809523814]

## Predicting on Kaggle Dataset

In [13]:
path = "C:\work\ml\digit_test_2.csv"
df = pd.read_csv(path)

test_data = np.array([np.array([x.reshape(-1, 28)], ) for x in np.array(df)])

In [25]:
prediction = model.predict(test_data)
Label = prediction.argmax(1)
ImageId = list(range(1, 28001))

In [26]:
d = {'ImageId': ImageId, 'Label': Label}
df = pd.DataFrame(data=d, index=None)
df.to_csv("submission.csv", index=False)